In [ ]:
!pip install optax

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
import optax
import flax.linen as nn
from typing import Tuple

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-25 00:17:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2024-08-25 00:17:56 (114 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [ ]:
# Hyperparameters
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [ ]:
# Data loading and preprocessing
def load_data():
    with open('input.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
    data = jnp.array(encode(text), dtype=jnp.int32)
    n = int(0.9 * len(data))
    train_data = data[:n]
    val_data = data[n:]
    return train_data, val_data, encode, decode, vocab_size


In [ ]:
train_data, val_data, encode, decode, vocab_size = load_data()

In [ ]:
def get_batch(data, key):
    data_size = data.shape[0]
    key, subkey = jax.random.split(key)
    ix = jax.random.randint(subkey, (batch_size,), 0, data_size - block_size)
    x = jnp.take(data, jnp.arange(block_size)[None, :] + ix[:, None], axis=0)
    y = jnp.take(data, jnp.arange(1, block_size + 1)[None, :] + ix[:, None], axis=0)
    return x, y, key

In [ ]:
class Head(nn.Module):
    head_size: int

    @nn.compact
    def __call__(self, x, training: bool = False):
        B, T, C = x.shape
        key = nn.Dense(features=self.head_size, use_bias=False)(x)
        query = nn.Dense(features=self.head_size, use_bias=False)(x)
        value = nn.Dense(features=self.head_size, use_bias=False)(x)

        tril = jnp.tril(jnp.ones((T, T)))

        wei = (query @ key.transpose(0, 2, 1)) * (self.head_size ** -0.5)
        wei = jnp.where(tril == 0, float('-inf'), wei)
        wei = jax.nn.softmax(wei, axis=-1)
        wei = nn.Dropout(rate=dropout)(wei, deterministic=not training)

        out = wei @ value
        return out

class MultiHeadAttention(nn.Module):
    num_heads: int
    head_size: int

    @nn.compact
    def __call__(self, x, training: bool = False):
        heads = [Head(self.head_size) for _ in range(self.num_heads)]
        out = jnp.concatenate([head(x, training) for head in heads], axis=-1)
        out = nn.Dense(features=n_embd)(out)
        out = nn.Dropout(rate=dropout)(out, deterministic=not training)
        return out

class FeedForward(nn.Module):
    @nn.compact
    def __call__(self, x, training: bool = False):
        x = nn.Dense(features=4 * n_embd)(x)
        x = nn.relu(x)
        x = nn.Dense(features=n_embd)(x)
        x = nn.Dropout(rate=dropout)(x, deterministic=not training)
        return x

class Block(nn.Module):
    @nn.compact
    def __call__(self, x, training: bool = False):
        x = x + MultiHeadAttention(num_heads=n_head, head_size=n_embd // n_head)(nn.LayerNorm()(x), training)
        x = x + FeedForward()(nn.LayerNorm()(x), training)
        return x

class GPTLanguageModel(nn.Module):
    vocab_size: int

    @nn.compact
    def __call__(self, idx, targets=None, training: bool = False):
        B, T = idx.shape

        token_emb = nn.Embed(num_embeddings=self.vocab_size, features=n_embd)(idx)
        pos_emb = nn.Embed(num_embeddings=block_size, features=n_embd)(jnp.arange(T))
        x = token_emb + pos_emb

        for _ in range(n_layer):
            x = Block()(x, training)

        x = nn.LayerNorm()(x)
        logits = nn.Dense(features=self.vocab_size)(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            # loss = jax.nn.sparse_categorical_crossentropy(logits, targets, from_logits=True).mean()
            loss = optax.softmax_cross_entropy_with_integer_labels(logits, targets).mean()


        return logits, loss

    def generate(self, params, idx, max_new_tokens, key):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self.apply(params, idx_cond, training=False, rngs={'dropout': key})
            logits = logits[:, -1, :]
            probs = jax.nn.softmax(logits, axis=-1)
            idx_next = jax.random.categorical(key, probs, axis=-1)
            idx = jnp.concatenate([idx, idx_next[:, None]], axis=1)
            key, _ = jax.random.split(key)
        return idx



In [ ]:
# Initialize the model
vocab_size = 65  # Example value, adjust as needed
model = GPTLanguageModel(vocab_size=vocab_size)
key = jax.random.PRNGKey(0)
params = model.init(key, jnp.ones((1, block_size), dtype=jnp.int32))

# Optimizer setup
optimizer = optax.adam(learning_rate)
opt_state = optimizer.init(params)

# Training loop
@jax.jit
def train_step(params, opt_state, x, y):
    def loss_fn(params):
        _, loss = model.apply(params, x, y, training=True)
        return loss

    loss, grads = jax.value_and_grad(loss_fn)(params)
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss

@jax.jit
def loss_fn(params, x, y):
    _, loss = model.apply(params, x, y, training=True)
    return loss

@jax.jit
def eval_step(params, x, y):
    _, loss = model.apply(params, x, y, training=False)
    return loss

def estimate_loss(params, key):
    losses = {split: [] for split in ['train', 'val']}
    for split in ['train', 'val']:
        data = train_data if split == 'train' else val_data
        for _ in range(eval_iters):
            key, subkey = jax.random.split(key)
            x, y, key = get_batch(data, subkey)
            loss = eval_step(params, x, y)
            losses[split].append(loss)
    return {k: jnp.mean(jnp.array(v)) for k, v in losses.items()}, key

# Main training loop
key = jax.random.PRNGKey(1)  # New key for training
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses, key = estimate_loss(params, key)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    key, subkey = jax.random.split(key)
    x, y, key = get_batch(train_data, subkey)
    loss = loss_fn(params, x, y)
    grads = jax.grad(loss_fn)(params, x, y)
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)



In [ ]:
# Generation
gen_key = jax.random.PRNGKey(2)
context = jnp.zeros((1, 1), dtype=jnp.int32)
generated = model.generate(params, context, max_new_tokens=500, key=gen_key)
print(decode(generated[0].tolist()))